In [31]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms
import torchvision
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

In [7]:
seed = 33 
test_train_ratio = 0.2
data_path = "images"

#checks if cuda is available   if so it is used
print("Cuda availability: "+str(torch.cuda.is_available()))
if torch.cuda.is_available():
    print("Using GPU")
    dev = "cuda:0"
    torch.cuda.set_device(dev)
else:
    print("Using CPU")
    dev = "cpu"


Cuda availability: False
Using CPU


In [9]:
def get_train_and_validation_data_loader(data_path="images", validation_split_ratio=0.1, seed=42, size=None):
    if size is None:
        size = [160, 120]
    data = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([
            transforms.Resize((size[0], size[1])),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    )
    dataset_size = len(data)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split_ratio * dataset_size))
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(
        data,
        num_workers=4,
        batch_size=1,
        sampler=train_sampler
    )
    valid_loader = DataLoader(
        data,
        num_workers=4,
        batch_size=1,
        sampler=valid_sampler
    )
    return train_loader, valid_loader

In [16]:
    train_loader, valid_loader = get_train_and_validation_data_loader(data_path, test_train_ratio, seed)


In [34]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 48, 3, padding=1)
        # linear layers
        self.fc1 = nn.Linear(48 * 40 * 30, 4096)
        self.fc2 = nn.Linear(4096, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 10)
        # dropout
        self.dropout = nn.Dropout(p=0.2)
        # max pooling
        self.pool = nn.MaxPool2d(2, 2)
        # Define relu activation and LogSoftmax output
        self.LogSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flattening the image
        x = x.view(x.size(0), -1)
        # linear layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.LogSoftmax(self.fc4(x))
        return x


In [ ]:
model = neural_net.NeuralNet()
model.to(dev)
#The negative log likelihood loss.
lossFunction = nn.NLLLoss()
#Stochastic Gradient Descent
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



In [40]:
num_epochs = 10
for epoch in range(num_epochs):
    loss_ = 0
    for image, label in train_loader:
        image, label = images.to(dev), labels.to(dev)
        # Forward Pass
        output = model(image)
        # Loss at each iteration by comparing to target(label)
        loss = lossFunction(output, label)
        # Backpropogating gradient of loss
        optimizer.zero_grad()
        loss.backward()

        # Updating parameters(weights and bias)
        optimizer.step()

        loss_ += loss.item()
    print("Epoch{}, Training loss:{}".format(epoch, loss_ / len(train_loader)))


hi
hi
hi
hi
hi
hi
hi
hi


KeyboardInterrupt: 

In [ ]:
torch.save(model, 'nn_v9.pt')
model = torch.load("nn_v9.pt")
    with torch.no_grad():
        correct = 0
        total = 0
        for image, label in valid_loader:
            image, label = images.to(dev), labels.to(dev)
            out = model(images)
            _, predicted = torch.max(out, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return print('Testing accuracy: {} %'.format(100 * correct / total))